In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from random import choice, randint
from pathlib import Path

import importlib
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle

import ptwt, pywt

from sphfile import SPHFile
import librosa
import librosa.display
import soundfile

import dataset
from dataset import pickle_stft_data

In [2]:
importlib.reload(dataset)

<module 'dataset' from '/home/pakumar/teams/ece251c-team-11/code/dataset.py'>

In [3]:
TRAIN_BATCH_SIZE = 1
TEST_BATCH_SIZE = 1

BASE_DIR = Path(os.getcwd()).parents[0]
DATA_DIR = BASE_DIR / 'data'

In [4]:
def load_pkl_data(split, num_samples):
    pkl_file = DATA_DIR / 'pkl_files' / '{}_data'.format(split) / 'clean_mixed_data_{}.pickle'.format(num_samples)
    if not pkl_file.is_file():
        print('Generating data ...')
        signal_len_lst = pickle_stft_data(split, num_samples=num_samples)
        plot_signal_stats(signal_len_lst)
        
    with open(pkl_file, 'rb') as handle:
        clean_mixed_data_dict = pickle.load(handle)
    
    return clean_mixed_data_dict

In [5]:
class SpeechDataset(Dataset):
    def __init__(self, split='train', num_samples=1000, alpha=0.8):
        assert split in ['train', 'val', 'test'], "Invalid split"
        self.split = split
        self.alpha = alpha
        self.num_samples = num_samples
        self.clean_mixed_data_dict = load_pkl_data(split, num_samples)
    
    def __len__(self):
        return self.num_samples
#         return len(self.file_list)
    
    def __getitem__(self, idx):
        stft_clean = self.clean_mixed_data_dict['clean'][idx]
        stft_mixed = self.clean_mixed_data_dict['mixed'][idx]
        
        return stft_clean, stft_mixed

In [7]:
test_dataset = SpeechDataset('test', num_samples=500)
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, collate_fn=dataset.custom_collate_fn)
print("Test dataset size:", len(test_dataloader))
# dataset_stats(test_dataloader)

Test dataset size: 500


In [8]:
model_save_path = os.path.join(BASE_DIR, "trained_model_ckpts/checkpoint.pth.tar")
model_checkpoint = torch.load(f=model_save_path)
# model = models.get_architecture(model_checkpoint["arch"], args.dataset_name, device)
model.load_state_dict(model_checkpoint['state_dict'])

NameError: name 'model' is not defined